In [1]:
import pandas as pd
import urllib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import time
import PIL
from PIL import Image

from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [2]:
import torch
from google.colab.patches import cv2_imshow
from imutils import perspective
import random

In [3]:
import functools
import re
import cvxpy as cp

In [4]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 14461, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 14461 (delta 58), reused 64 (delta 39), pack-reused 14371
Receiving objects: 100% (14461/14461), 13.63 MiB | 15.68 MiB/s, done.
Resolving deltas: 100% (9959/9959), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 5.1 MB/s 
     |████████████████████████████████| 62 kB 787 kB/s 
     |████████████████████████████████| 1.6 MB 48.2 MB/s 


In [5]:
!git clone https://github.com/goal2002man/USD_Computer_Vision_Final/

Cloning into 'USD_Computer_Vision_Final'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 15), reused 23 (delta 7), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [10]:
!git clone -b model_weights https://github.com/goal2002man/USD_Computer_Vision_Final/

Cloning into 'USD_Computer_Vision_Final'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 15), reused 23 (delta 7), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [11]:
!ls USD_Computer_Vision_Final

best_char.pt	       environment.yml	YOLOv5_License_Plate_Detector.ipynb
best_lp.pt	       README.md
character_recognition  yolov5


In [16]:
# License Plate Detector
path_yolo_best_weights = '/content/USD_Computer_Vision_Final/best_lp.pt'

lpd_model = torch.hub.load('yolov5/', 'custom', path=path_yolo_best_weights, source='local')  # local repo

YOLOv5 🚀 v7.0-30-g342fe05 Python-3.8.16 torch-1.13.0+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [15]:
# Charachter Segmentation
path_yolo_best_weights_char_segmenter = '/content/USD_Computer_Vision_Final/best_char.pt' 

char_segm_model = torch.hub.load('yolov5/', 'custom', path=path_yolo_best_weights_char_segmenter, source='local')  # local repo

YOLOv5 🚀 v7.0-30-g342fe05 Python-3.8.16 torch-1.13.0+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [14]:
def char_segmentation_yolo(file_name):
 input_images = cv2.imread(file_name)
 results = lpd_model(input_images)
 boxes = results.pandas().xyxy[0]  
 cv2_imshow(input_images)

 # Identify index of license plate bounding box
 index_lp = 10000
 flag = 0
 for i in range(boxes.shape[0]):
   if boxes.loc[i].at["name"] == 'license-plate':
     index_lp = i
     flag = 1
     break
 if flag == 0:
     display ('No license plate detected')
     return []

 # Coordinates of bounding box with highest confidence interval
 bb_xmin = boxes.loc[index_lp].at["xmin"]
 bb_xmax = boxes.loc[index_lp].at["xmax"]
 bb_ymin = boxes.loc[index_lp].at["ymin"]
 bb_ymax = boxes.loc[index_lp].at["ymax"]

 # # Image patch for character segmentation
 img_for_char_segm = input_images[round(bb_ymin):round(bb_ymax), round(bb_xmin):round(bb_xmax)]
 cv2_imshow(img_for_char_segm)

 # Charachter segmentation
 results_segm = char_segm_model(img_for_char_segm)
 boxes_segm = results_segm.pandas().xyxy[0]
 print(boxes_segm)
 
 ## Display bounding boxes for characters and add the cropped charachters to a list
 char_images = []
 img_for_char_segm_copy = img_for_char_segm.copy()
 for i in range(boxes_segm.shape[0]):
   if boxes_segm.loc[i].at["name"] == 'character':
     bb_segm_xmin = boxes_segm.loc[i].at["xmin"]
     bb_segm_xmax = boxes_segm.loc[i].at["xmax"]
     bb_segm_ymin = boxes_segm.loc[i].at["ymin"]
     bb_segm_ymax = boxes_segm.loc[i].at["ymax"]
     crop_char = img_for_char_segm[round(bb_segm_ymin):round(bb_segm_ymax), round(bb_segm_xmin):round(bb_segm_xmax)]
     cv2_imshow(cv2.resize(crop_char,(100,100)))
     char_images.append(crop_char)
     cv2.rectangle(img_for_char_segm_copy, (int(bb_segm_xmin),int(bb_segm_ymin)), (int(bb_segm_xmax),int(bb_segm_ymax)), (255,0,0), 1)

 cv2_imshow(cv2.resize(img_for_char_segm_copy,(300,100)))

 return input_images, img_for_char_segm, char_images

In [ ]:
# Call the function
filename = ''  # UPDATE with your file name
input_images, img_for_char_segm, char_images =  char_segmentation_yolo(filename)
